In [54]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle

In [55]:
## load the train model, scaler pickel, ohe

model = load_model('model.h5')

# load the encoder and scaler
with open('lable_encoder_gender.pkl','rb') as file:
    lable_encoder_gender=pickle.load(file)
with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)
with open('geo_encoder.pkl','rb') as file:
    geo_encoder=pickle.load(file)


In [56]:
input_data={
    'CreditScore':720,
    'Gender':'Male',
    'Geography':'Spain',
    'Age':25,
    'Tenure':4,
    'Balance':756463,
    'NumOfProducts':1,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':76045,
}

In [57]:
# ohe encode geography
geo_encoded = geo_encoder.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded ,columns=geo_encoder.get_feature_names_out(['Geography']))


d:\DL\ANN\Projects\End to End CHURN model using ANN\venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [58]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Gender,Geography,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,720,Male,Spain,25,4,756463,1,1,1,76045


In [59]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,0.0,0.0,1.0


In [60]:
#  encode categorical variable

input_df['Gender']=lable_encoder_gender.transform(input_df['Gender'])

In [61]:
# concatination ohe
input_df=pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df],axis=1)
   

In [62]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,720,1,25,4,756463,1,1,1,76045,0.0,0.0,1.0


In [63]:
# scale values

input_scaled=scaler.transform(input_df)
input_scaled

array([[ 0.75714403,  0.90911166, -1.35286146, -0.3574909 , 10.92170815,
        -0.89163935,  0.63367318,  0.95214374, -0.39475128, -1.02020406,
        -0.57581067,  1.77464858]])

In [64]:
# predict churn
prediction=model.predict(input_scaled)
input_scaled

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step


array([[ 0.75714403,  0.90911166, -1.35286146, -0.3574909 , 10.92170815,
        -0.89163935,  0.63367318,  0.95214374, -0.39475128, -1.02020406,
        -0.57581067,  1.77464858]])

In [65]:
prediction_proba= prediction[0][0]

In [66]:
prediction_proba

np.float32(0.12698999)

In [67]:
if prediction_proba < 0.5:
    print("The customer is likely to be churned!")
else:
    print("The customer is likely to continue!")

The customer is likely to be churned!
